In [ ]:
!pip install pytorch-pretrained-bert
!pip install seqeval
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam, BertForSequenceClassification,BertModel



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
with open("corpus.txt", 'w') as f:
    
    for token in tokenizer.vocab.keys():
                 
        f.write(token + '\n')

In [ ]:
import torch

if torch.cuda.is_available():    
    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df=pd.read_csv('/content/drive/My Drive/tweets.csv',delimiter=',',encoding='ISO-8859-1',low_memory=False)

In [ ]:
input_ids = []
token_text = []
tweetdf = df[0:10000]
df2 = df[810000:820000]
tweetdf = tweetdf.append(df2)
tweetdf.columns = ['sentiment','id','date','query','uname','tweet']
print(len(tweetdf))
sentences = tweetdf.tweet.values
labels = tweetdf.sentiment.values

for sent in sentences:
    tokenized_text = tokenizer.tokenize(sent)
    token_text.append(tokenized_text)
    encoded_sent = tokenizer.encode(
                        sent,                      
                        add_special_tokens = True,
                        max_length = 512 
                   )

    input_ids.append(encoded_sent)



In [ ]:
print('Original: ', sentences[11])
print('Labels: ', labels)
print('Token IDs:', input_ids[11])
print('tokenized text: ',token_text[11])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 64

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')

In [ ]:
attention_masks = []

for sent in input_ids:

    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)



In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [ ]:
train_inputs = torch.LongTensor(train_inputs)
validation_inputs = torch.LongTensor(validation_inputs)

train_labels = torch.LongTensor(train_labels)
validation_labels = torch.LongTensor(validation_labels)

train_masks = torch.LongTensor(train_masks)
validation_masks = torch.LongTensor(validation_masks)

print(train_inputs[11])
print(train_masks[11])


In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
#train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)



valid_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=16)

print(len(train_dataloader))

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
model = model.cuda()


In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [ ]:
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
!CUDA_LAUNCH_BLOCKING=1

In [ ]:
import torch.nn as nn
t = [] 
loss_ot = []
accuracy_ot = []

# Store our loss and accuracy for plotting
train_loss_set = []

epochs = 5
max_grad_norm = 1.0

for epoch in tqdm(range(epochs)):
    # TRAIN loop
      model.train()
  
  # Tracking variables
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
      for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)


    # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
    # Forward pass
        criterion = nn.CrossEntropyLoss()
        outputs = model(b_input_ids)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs,b_labels)
        train_loss_set.append(loss.item())    
    # Backward pass
        loss.backward()
    # Update parameters and take a step using the computed gradient
        optimizer.step()

        #print("Iteration: {}/{} | Loss: {}".format(step,len(train_dataloader),loss))
    
    
    # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
      loss_ot.append(tr_loss/nb_tr_steps)
      #train_loss = tr_loss/nb_tr_steps
      print("Training Loss: {}".format(tr_loss/nb_tr_steps))
      #print("Epoch {}/{} | Train loss: {}".format(epoch+1,epochs,train_loss))
    
    # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
      model.eval()

  # Tracking variables 
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
      for batch in valid_dataloader:
    # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
      # Forward pass, calculate logit predictions
          logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
      accuracy_ot.append(eval_accuracy/nb_eval_steps)
      #val_accuracy = eval_accuracy/nb_eval_steps
    
    
    
      #print("Train Loss: {} | Validation Accuracy: {}".format(train_loss,val_accuracy))
      print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

torch.save(model,"/content/drive/My Drive/bertmodel1.pth")

import matplotlib.pyplot as plt 
epochs_ot = list(range(0,epochs))
# plotting the points  
plt.plot(epochs_ot, accuracy_ot) 
  
# naming the x axis 
plt.xlabel('Epochs') 
# naming the y axis 
plt.ylabel('Accuracy') 
  
# giving a title to my graph 
plt.title('Epochs vs Accuracy!') 
  
# function to show the plot 
plt.show() 

plt.plot(epochs_ot,loss_ot)
plt.xlabel('Epochs') 
# naming the y axis 
plt.ylabel('Loss') 
  
# giving a title to my graph 
plt.title('Epochs vs Loss!') 
  
# function to show the plot 
plt.show() 

In [ ]:
import torch
from google.colab import drive
drive.mount('/content/drive')
model = torch.load("/content/drive/My Drive/bertmodel1.pth")
model.eval()



In [ ]:
import pandas as pd

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1)[:, None])
    return e_x / np.sum(e_x, axis=1)[:, None]


def keywordextract(text,filename):
  tkns = tokenizer.tokenize(text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
  segments_ids = [0] * len(tkns)
  tokens_tensor = torch.tensor([indexed_tokens]).to(device)
  segments_tensors = torch.tensor([segments_ids]).to(device)
  label_list = ['negative','positive']
  label_dict = {0: 'negative' , 1: 'positive'}
  result = pd.DataFrame(columns=['sentence','logit','prediction','sentiment_score'])
  model.eval()
  logits = model(tokens_tensor, token_type_ids=None,attention_mask=segments_tensors)
  logits = logits.detach().cpu().numpy()
  logits = softmax(logits)
  sentiment_score = pd.Series(logits[:,0] - logits[:,1])
  predictions = np.squeeze(np.argmax(logits, axis=1))
  print("Text:"+text)
  print("Sentiment Score:")
  print(sentiment_score)
  print("Predictions:['Negative:0','Positive:1']:")
  print(predictions)
  print("Logits:['Negative','Positive']:")
  print(logits)
  res = {"text":text,"sentiment":[predictions]}
  df = pd.DataFrame(res)
  df.to_csv('/content/drive/My Drive/'+filename+'.csv', mode='a', header=False)

 
    
          

In [ ]:
line = "I hate it when any athlete appears to tear an ACL on live television."
keywordextract(line,"sample")

In [ ]:
!pip install rake-nltk

import operator

from rake_nltk import Metric,Rake

phrases = []
def keyphrases_extract(text):
  r = Rake(ranking_metric=Metric.WORD_DEGREE,min_length=1) 
  r.extract_keywords_from_text(text)
  p_list = r.get_ranked_phrases()
  p = {'text':text,'phrase':p_list}
  pdf = pd.DataFrame(p)
  pdf.to_csv('/content/drive/My Drive/phrases.csv', mode='a', header=False)


In [ ]:
import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

auth = tweepy.AppAuthHandler('i2PBoMdhbQXqmIdAwXZKgawWQ', 'LYxmHirTinFxIMZbh1O3pkHPpIvo3MnLraCQijWx5N91n2sCAZ')

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

#@title Twitter Search API Inputs
#@markdown ### Enter Search Query:
searchQuery = 'oneplus nord' #@param {type:"string"}

maxTweets = 20
Filter_Retweets = True #@param {type:"boolean"}

tweetsPerQry = 100  # this is the max the API permits
tweet_lst = []

if Filter_Retweets:
  searchQuery = searchQuery + ' -filter:retweets'  # to exclude retweets

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -10000000000

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, lang="en")
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        lang="en", max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found")
            break
        for tweet in new_tweets:
          if hasattr(tweet, 'reply_count'):
            reply_count = tweet.reply_count
          else:
            reply_count = 0
          if hasattr(tweet, 'retweeted'):
            retweeted = tweet.retweeted
          else:
            retweeted = "NA"
            
          # fixup search query to get topic
          topic = searchQuery[:searchQuery.find('-')].capitalize().strip()
          
          # fixup date
          tweetDate = tweet.created_at.date()
          
          tweet_lst.append([tweetDate, topic, 
                      tweet.id, tweet.user.screen_name, tweet.user.name, tweet.text, tweet.favorite_count, 
                      reply_count, tweet.retweet_count, retweeted])

        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("some error : " + str(e))
        break

clear_output()
print("Downloaded {0} tweets".format(tweetCount))

In [ ]:
pd.set_option('display.max_colwidth', -1)

# load it into a pandas dataframe
tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'topic', 'id', 'username', 'name', 'tweet', 'like_count', 'reply_count', 'retweet_count', 'retweeted'])
tweet_df.head()
tweet_df.to_csv('/content/drive/My Drive/opntwweets.csv')


In [ ]:
for index, row in tqdm(tweet_df.iterrows(), total=tweet_df.shape[0]):
  cleanedTweet = row['tweet'].replace("#", "")
  keywordextract(cleanedTweet,searchQuery)
  keyphrases_extract(cleanedTweet)



In [ ]:
!pip install matplotlib
!pip install pandas
!pip install wordcloud

from google.colab import drive
drive.mount('/content/drive')

from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 

comment_words = ' '
stopwords = set(STOPWORDS)

phrase_df=pd.read_csv('/content/drive/My Drive/phrases.csv',encoding='utf-8',low_memory=False,index_col=1)
phrase_df.columns = ['phrase']
phrase_df = phrase_df.groupby('phrase')
print(phrase_df)
phrase_list = phrase_df
for val in phrase_list: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
          
    for words in tokens: 
      comment_words = comment_words + words + ' '
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 6).generate(comment_words) 


from google.colab import files

wordcloud.to_file('/content/drive/My Drive/wordcloud.png')
files.download('/content/drive/My Drive/wordcloud.png')
# plot the WordCloud image                        
plt.figure(figsize = (6, 6), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
import pandas as pd

sent_df=pd.read_csv('/content/drive/My Drive/'+searchQuery+'.csv',encoding='utf-8',low_memory=False)
sent_df.columns = ['rand','tweet','sent']

x = sent_df.tweet.values
y = sent_df.sent.values

pos_tweets = [ tweet for index, tweet in enumerate(sent_df['tweet']) if sent_df['sent'][index] == 1]
neg_tweets = [ tweet for index, tweet in enumerate(sent_df['tweet']) if sent_df['sent'][index] == 0]

print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(sent_df['tweet'])))
print("Percentage of negative tweets: {}%".format(len(neg_tweets)*100/len(sent_df['tweet'])))

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sentiment_summary = dict(pos=0,neg=0,neu=0)

for sentence in x:
  sentiment_score = sid.polarity_scores(sentence)
  if sentiment_score["compound"] == 0.0:
            sentiment_summary["neu"] += 1
  elif sentiment_score["compound"] > 0.0:
            sentiment_summary["pos"] += 1
  else:
            sentiment_summary["neg"] += 1

x=('positive','negative','neutral')
y_pos = np.arange(len(x))
y=[sentiment_summary['pos'],sentiment_summary['neg'],sentiment_summary['neu']]

print(sentiment_summary)

plt.bar(x, y, align='center', alpha=0.5)
plt.xticks(y_pos,x)
plt.ylabel('sentiment balance')
plt.title('Sentiment Balance')

plt.show()